In [1]:
import time
import os
from google.cloud import bigquery
from google.oauth2 import service_account

from dotenv import load_dotenv
load_dotenv()
bigquery_creds = os.getenv("SERVICE_ACCOUNT_CREDS")
credentials = service_account.Credentials.from_service_account_file(bigquery_creds)
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [2]:
from google.cloud import bigquery

def get_bigquery_table_schema_for_json(project_id, dataset_id, table_id):
    # Construct the full table ID
    table_ref = f"{project_id}.{dataset_id}.{table_id}"

    # Get the table
    table = client.get_table(table_ref)  # API Request

    # Convert schema to JSON-like structure
    schema_json = []
    for field in table.schema:
        field_dict = {
            "name": field.name,
            "type": field.field_type,
            "mode": field.mode
        }
        # Add nested fields if the field is a RECORD type (for nested structures)
        if field.field_type == "RECORD":
            field_dict["fields"] = convert_subfields(field.fields)

        schema_json.append(field_dict)

    return schema_json

def convert_subfields(fields):
    subfields_json = []
    for subfield in fields:
        subfield_dict = {
            "name": subfield.name,
            "type": subfield.field_type,
            "mode": subfield.mode
        }
        if subfield.field_type == "RECORD":
            subfield_dict["fields"] = convert_subfields(subfield.fields)
        subfields_json.append(subfield_dict)
    return subfields_json

# Example usage
project_id = 'kevinesg-production'
dataset_id = 'raw'
table_id = 'ledger'

schema_json_format = get_bigquery_table_schema_for_json(project_id, dataset_id, table_id)

# schema_json_format is now ready to be used with client.schema_from_json()


In [4]:
schema_json_format

[{'name': 'id', 'type': 'STRING', 'mode': 'NULLABLE'},
 {'name': 'year', 'type': 'INTEGER', 'mode': 'NULLABLE'},
 {'name': 'month', 'type': 'INTEGER', 'mode': 'NULLABLE'},
 {'name': 'day', 'type': 'INTEGER', 'mode': 'NULLABLE'},
 {'name': 'item', 'type': 'STRING', 'mode': 'NULLABLE'},
 {'name': 'type', 'type': 'STRING', 'mode': 'NULLABLE'},
 {'name': 'cost', 'type': 'FLOAT', 'mode': 'NULLABLE'},
 {'name': 'to', 'type': 'STRING', 'mode': 'NULLABLE'},
 {'name': 'store', 'type': 'STRING', 'mode': 'NULLABLE'},
 {'name': 'source', 'type': 'STRING', 'mode': 'NULLABLE'},
 {'name': 'created_at', 'type': 'STRING', 'mode': 'NULLABLE'},
 {'name': 'updated_at', 'type': 'STRING', 'mode': 'NULLABLE'}]